# Steps for the execution of the program: 
 1. Importing the modules.
 2. Creation of the descriptive table.   
 3. Cleaning and preproccesing data. 
  
   3.1. Delete the variables with a determined percent of missing values.
   
   3.2. Transform categorical data.

###  1. Importing the modules.

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

### 2. Creation of the descriptive table.

#### 2.1. Reading the files.

In [3]:
mypath = 'C:/Users/miguelico/Google Drive/master/TFM/Data/Variables'
mypath_out = 'C:/Users/miguelico/Google Drive/master/TFM/Data/Variables limpiadas'

#### 2.2. Implementation of the function fill_table.
This function provides a DataFrame with the most important aspect of our dataset. Here, you can see the list of variables:

 - **Variable**: This column represents the name of the column.
 - **Number of rows**: This column represent the umber of rows of the column. The number is the same for all variables of a same table.
 - **Number of different id**: This variable represents the number of different's ids that Variable has in a table.
 - **Number of NaN**: This variable represents the number of the missing values which there are in each column.
 - **Number of different values**: This column represents the number of different values which are in each column.
 - **Number of PD**: This column represents the number of Patients with Parkinson (PwP), we calculate this number by using the fox_insight_id and the table Users.
 - **Number of EarlyPD**: This column represents the number of PwP who have the disease for a period less than 3 years, we calculate this number by using the fox_insight_id and the table Users.
 - **Number of MidPD**: This column represents the number of PwP who have the disease for a period between 3 years and 10 years, we calculate this number by using the fox_insight_id and the table Users.
 - **Number of LatePD**: This column represents the number of PwP who have the disease for a period more than 10 years, we calculate this number by using the fox_insight_id and the table Users.
 - **Name of table**: This column represents the name of table where the column belongs.
 - **dtype**: This column represents the type of variable.


In [4]:
#Esta función tiene como inputs la carpeta donde se encuentra las tablas del dataset y el nombre 
#que se le quiere dar al archivo, además de la carpeta de destino donde guardaremos la tabla y 
#la ruta de la tabla fox_insight_variables
def fill_table(folder, file_name, folder_export, fox_insight_variables_root):
#Guarda en la variable files el nombre de los archivos que hay en la carpeta
    files = [f for f in listdir(folder) if isfile(join(folder, f))]
    files = [f.replace('.csv', '') for f in files]
#Cargamos la variable User que usaremos para rellenar el nombre de las columnas de PD
    Users = pd.read_csv(folder + "/Users.csv" , sep = ",")
#Creamos la tabla descriptiva con el nombre df
    df = pd.DataFrame(columns=['Variable', 'Number of rows', 'Number of different id', 'Number of NaN', 'Number of different values', 'Number of PD', 'Number of EarlyPD', 
                               'Number of MidPD', 'Number of LatePD', 'Name of table', 'dtype'])
#creamos una tabla auxiliar que donde se guardará la información
    df_aux = pd.DataFrame(columns= list(df.columns))
#Para cada tabla de la carpeta creamos un dataframe que llamaremos df1
    for field in files:
        root = folder + "/" + field + ".csv"
        df1 = pd.read_csv(root , sep = ",")
#Rellenamos df_aux con la información de cada tabla
        df_aux['Variable'] = list(df1.columns)
        df_aux['Number of rows'] = df1.shape[0]
        df_aux['Number of different id'] = len(df1['fox_insight_id'].unique())
        num_nan = []
        [num_nan.append(df1[x].isna().sum()) for x in list(df1.columns)]
        df_aux['Number of NaN'] = num_nan
        df_aux['Number of different values'] = [len(df1[x][df1[x].notna()].unique()) for x in df1.columns]
        df_aux['Name of table'] = field
        df_aux['dtype'] = [x for x in df1.dtypes.to_frame().reset_index()[0]]
#Si la tabla no es la de Users importamos unimos dicha tabla con Users para obtener la información sobre PD
        if field != 'Users':
            df1 = df1.merge(Users[['fox_insight_id', 'InitPDDiag', 'YearsWithPD']], how = 'inner', on = 'fox_insight_id')
        df_aux['Number of PD'] = len(df1['fox_insight_id'][df1['InitPDDiag'] == 1].unique())
        df_aux['Number of EarlyPD'] = len(df1['fox_insight_id'][df1['YearsWithPD'] == 0].unique())
        df_aux['Number of MidPD'] = len(df1['fox_insight_id'][df1['YearsWithPD'] == 1].unique())
        df_aux['Number of LatePD'] = len(df1['fox_insight_id'][df1['YearsWithPD'] == 2].unique())
#Añadimos la información a df mediante la función append
        df = df.append(df_aux)
#Eliminamos todas las filas de la tabla df_aux y volvemos a iniciar el bucle
        df_aux = df_aux[0:0]
#Eliminamos los duplicados
    df_aux = df_aux.drop_duplicates(df)
#Importamos la tabla fox_insight_variables
    Variables = pd.read_csv(fox_insight_variables_root , sep = ",")
    Variables = Variables.rename(columns = {'variable': 'Variable'})
#Unimos dicha columna a nuestro dataframe
    df = df.merge(Variables, how = 'inner', on = 'Variable')
#Eliminamos los duplicados
    df = df.drop_duplicates()
#Creamos la columna longitudinal que nos dice si la columna de la tabla es longitudinal o no
    df['Longitudinal'] = ["No" if df['Number of rows'].iloc[i] == df['Number of different id'].iloc[i] else 'Yes' for i in range(df.shape[0])]
#Eliminamos las columnas de Users que hemos ido uniendo a las distintas tablas 
    df = df.drop(df[(df['Name of table'] != 'Users') & ((df['Variable'] == 'InitPDDiag') | (df['Variable'] == 'YearsWithPD'))].index, axis = 0)
    df = df.reset_index()
    df = df.drop('index', axis = 1)
#Guardamos nuestro dataframe a un csv con el nombre que nosotros elijamos a la carpeta de destino que queramos   
    df.to_csv(folder_export + "/" + file_name+ ".csv", index=False)
    return df

In [5]:
fill_table(mypath, 'Análisis descriptivo de las variables antes preproc', "C:/Users/miguelico/Google Drive/master/TFM/Data" ,
           "C:/Users/miguelico/Google Drive/master/TFM/Data/FoxInsightVariables (1).csv")

,Variable,Number of rows,Number of different id,Number of NaN,Number of different values,Number of PD,Number of EarlyPD,Number of MidPD,Number of LatePD,Name of table,dtype,variable_description,category,Longitudinal
0,Sex,177143,50565,47357,2,36471,16395,13877,5920,About,float64,What is your biological sex?,About You,Yes
1,HeightPNA,177143,50565,0,2,36471,16395,13877,5920,About,int64,Prefer not to answer,About You,Yes
2,HeightInch,177143,50565,48866,118,36471,16395,13877,5920,About,float64,In inches [Derived],About You,Yes
3,HeightCm,177143,50565,48866,122,36471,16395,13877,5920,About,float64,In centimeters [Derived],About You,Yes
4,WeightPNA,177143,50565,0,2,36471,16395,13877,5920,About,int64,Prefer not to answer,About You,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290,DiagWhoNeurPD,50607,50607,0,2,36501,16396,13877,5920,Users,int64,A general neurologist,Registration,No
5291,DiagWhoSpecPD,50607,50607,0,2,36501,16396,13877,5920,Users,int64,A neurologist specializing in movement disorde...,Registration,No
5292,DiagWhoOtherPD,50607,50607,0,2,36501,16396,13877,5920,Users,int64,Other type of physician or healthcare professi...,Registration,No
5293,InitReferralSource,50607,50607,5832,12,36501,16396,13877,5920,Users,float64,How did you hear about the Fox Insight study?,Registration,No


###  3. Cleaning and preproccesing data. 

#### 3.1. Delete the variables with a determined percent of missing values.

In [6]:
#Esta función tiene como input la tabla análisis descriptivo creada con la función fill_table, 
#la carpeta donde están las variables, la carpeta donde queremos guardar la tabla y finalmente el porcentaje 
#de valores a partir del cual se elimina
def delete_columns(folder_analisis_descriptive, folder_input, folder_output, perc):
    AD = pd.read_csv(folder_analisis_descriptive + ".csv", sep = ',') 
    for y in AD['Name of table'].unique():
    #cond1 filtra la tabla AD para cada tabla de la que queremos eliminar columnas
        cond1 = AD['Name of table'] == y
    #Importamos la tabla y a un dataframe
        root = folder_input + "/" + y + ".csv"
        df = pd.read_csv(root , sep = ",")
    #Para cada columna de la tabla y, si tiene un % mayor que perc la elimina de la tabla y
        for x in list(AD['Variable'][cond1].unique()):
            cond2 = AD['Variable'] == x
            if float(AD['Number of NaN'][cond1 & cond2]/AD['Number of rows'][cond1 & cond2])>=0.9:
                df = df.drop(x, axis = 1)
    #Guarda la tabla y actualizada en la carpeta que nosotros especifiquemos
        df.to_csv(folder_output + "/"+ y + ".csv", index=False)

In [7]:
delete_columns("C:/Users/miguelico/Google Drive/master/TFM/Data/Análisis descriptivo de las variables antes preproc"  , 
               mypath, mypath_out, 0.9)

In [8]:
fill_table(mypath_out, "Análisis descriptivo y explotario de las variables limpiadas", "C:/Users/miguelico/Google Drive/master/TFM/Data" ,
           "C:/Users/miguelico/Google Drive/master/TFM/Data/FoxInsightVariables (1).csv")

,Variable,Number of rows,Number of different id,Number of NaN,Number of different values,Number of PD,Number of EarlyPD,Number of MidPD,Number of LatePD,Name of table,dtype,variable_description,category,Longitudinal
0,Sex,177143,50565,47357,2,36471,16395,13877,5920,About,float64,What is your biological sex?,About You,Yes
1,HeightPNA,177143,50565,0,2,36471,16395,13877,5920,About,int64,Prefer not to answer,About You,Yes
2,HeightInch,177143,50565,48866,118,36471,16395,13877,5920,About,float64,In inches [Derived],About You,Yes
3,HeightCm,177143,50565,48866,122,36471,16395,13877,5920,About,float64,In centimeters [Derived],About You,Yes
4,WeightPNA,177143,50565,0,2,36471,16395,13877,5920,About,int64,Prefer not to answer,About You,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,DiagWhoNeurPD,50607,50607,0,2,36501,16396,13877,5920,Users,int64,A general neurologist,Registration,No
3319,DiagWhoSpecPD,50607,50607,0,2,36501,16396,13877,5920,Users,int64,A neurologist specializing in movement disorde...,Registration,No
3320,DiagWhoOtherPD,50607,50607,0,2,36501,16396,13877,5920,Users,int64,Other type of physician or healthcare professi...,Registration,No
3321,InitReferralSource,50607,50607,5832,12,36501,16396,13877,5920,Users,float64,How did you hear about the Fox Insight study?,Registration,No


#### 3.2. Creation of a table with all timeless variables.



El objetivo de esta parte es crear una tabla con la mayor cantidad de columnas.

In [26]:
def merge_tables():
    About = pd.read_csv("C:/Users/miguelico/Google Drive/master/TFM/Data/Variables limpiadas/About.csv" , sep = ",")
    About = About.drop(['HeightPNA', 'WeightPNA', 'HeightInch', 'WeightLbs'], axis = 1)
    About = About.drop_duplicates()
    Users = pd.read_csv("C:/Users/miguelico/Google Drive/master/TFM/Data/Variables limpiadas/Users.csv" , sep = ",")
    Users = Users.drop(['age', 'days_elapsed'], axis = 1)
    About = Users.merge(About, how = 'inner', on = 'fox_insight_id')
    About = About[['fox_insight_id', 'age', 'days_elapsed', 'days_acquired', 'schedule_of_activities',
                  'Sex','LocCountry', 'InitPDDiag', 'InitPDDiagAge',
       'AgeAtEnrollment', 'YearsWithPD', 'PDConnectFirstCTR',
       'PDConnectSecondCTR', 'PDConnectSpouseCTR', 'PDConnectFrienCTR',
       'PDConnectCareCTR', 'PDConnectPrevCTR', 'PDConnectGenetCTR',
       'PDConnectWorkCTR', 'PDConnectKnowCTR', 'PDConnectNoCTR',
       'PDConnectOtherCTR', 'DiagWhoDocPD', 'DiagWhoNeurPD', 'DiagWhoSpecPD',
       'DiagWhoOtherPD', 'InitReferralSource', 'InitReferralSourceDetail',
       'HeightCm', 'WeightKgs', 'RaceW', 'RaceAA', 'RaceAI', 'RaceA', 'RaceNH',
       'RacePNA', 'EthnNotHispanic', 'EthnMexican', 'EthnPuerto', 'EthnCuban',
       'EthnLatino', 'EthnPNA', 'Education', 'Income', 'Employment', 'Veteran',
       'Research']]
    Genetic = pd.read_csv("C:/Users/miguelico/Google Drive/master/TFM/Data/Variables limpiadas/Genetic.csv" , sep = ",")
    Genetic = Genetic.drop('platform_version', axis = 1)
    About = About.merge(Genetic, how = 'left', on = 'fox_insight_id')
    return About

In [27]:
clean_timeless_variables()

,fox_insight_id,age,days_elapsed,days_acquired,schedule_of_activities,Sex,LocCountry,InitPDDiag,InitPDDiagAge,AgeAtEnrollment,...,rs11158026,rs9261484,rs10513789,rs76904798,rs10797576,rs2230288,rs34637584,rs80356773,rs34424986,i4000415
0,FOX_000076,82.1,29889,0,REG,1.0,United States,1,81.3,82.1,...,NaN,CC,TT,NaN,NaN,CC,GG,CC,GG,TT
1,FOX_000076,83.1,30253,0,12,1.0,United States,1,81.3,82.1,...,NaN,CC,TT,NaN,NaN,CC,GG,CC,GG,TT
2,FOX_000076,84.1,30621,0,24,1.0,United States,1,81.3,82.1,...,NaN,CC,TT,NaN,NaN,CC,GG,CC,GG,TT
3,FOX_000076,85.1,30981,0,36,1.0,United States,1,81.3,82.1,...,NaN,CC,TT,NaN,NaN,CC,GG,CC,GG,TT
4,FOX_000087,63.3,23061,0,REG,2.0,United States,1,54.4,63.3,...,CT,CT,GT,CC,CC,CC,GG,CC,GG,TT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177138,FOX_999971,67.6,24606,0,REG,2.0,United States,1,64.1,67.6,...,CC,CC,TT,CC,CC,CC,GG,CC,GG,TT
177139,FOX_999971,68.6,24970,0,12,2.0,United States,1,64.1,67.6,...,CC,CC,TT,CC,CC,CC,GG,CC,GG,TT
177140,FOX_999971,69.6,25334,0,24,2.0,United States,1,64.1,67.6,...,CC,CC,TT,CC,CC,CC,GG,CC,GG,TT
177141,FOX_999976,63.8,23207,0,REG,1.0,United States,1,62.7,63.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
AcuteHealth = pd.read_csv("C:/Users/miguelico/Google Drive/master/TFM/Data/Variables limpiadas/AcuteHealth.csv" , sep = ",")

In [30]:
AcuteHealth

,fox_insight_id,age,days_elapsed,days_acquired,schedule_of_activities,AcuteHeart,AcuteStroke,AcuteTBI,AcuteSurgery,AcuteSurgeryTypeCar,AcuteSurgeryTypeOrt,AcuteSurgeryTypeGas,AcuteSurgeryTypeCra,AcuteSurgeryTypeTum,AcuteSurgeryTypePul,AcuteSurgeryTypeENT,AcuteSurgeryTypeEye,AcuteSurgeryTypeRep,AcuteSurgeryTypeCos,AcuteSurgeryTypeOth
0,FOX_000076,83.3,30345,0,15,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,FOX_000087,63.7,23212,0,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,FOX_000126,49.9,18134,0,4,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,FOX_000126,50.9,18498,0,16,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,FOX_000146,59.6,21721,0,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34717,FOX_999890,64.3,23389,0,27,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34718,FOX_999926,90.0,32795,0,28,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34719,FOX_999949,82.4,30056,0,15,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34720,FOX_999971,67.8,24697,0,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Users